In [1]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
model = load_model("adhd_eeg_model.h5")


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the original dataset
df = pd.read_csv("adhdata.csv")

# Encode class labels (optional if needed)
df['Class'] = df['Class'].map({'Control': 0, 'ADHD': 1})

# Drop rows with missing values if any
df.dropna(inplace=True)

# --- Prepare a sample for prediction ---
# Pick a single random row as new sample
new_df = df.sample(n=1, random_state=42)

# Save the label (optional: to compare actual vs predicted later)
true_label = new_df['Class'].values[0]

# Drop 'Class' to simulate test scenario (model doesn't see the answer)
new_df = new_df.drop(['Class'], axis=1)

# --- Preprocess for prediction ---
# Drop 'ID' if present
if 'ID' in new_df.columns:
    new_df = new_df.drop(['ID'], axis=1)

# Use the **same scaler used during training**
scaler = StandardScaler()
X_train_sampled = df.drop(['Class', 'ID'], axis=1)
scaler.fit(X_train_sampled)  # Fit scaler on training data

# Scale the new sample
new_data_scaled = scaler.transform(new_df)

# Reshape for model input (Conv1D expects 3D input)
new_data_reshaped = new_data_scaled.reshape((1, 1, new_data_scaled.shape[1]))

print("Prepared new sample shape:", new_data_reshaped.shape)

Prepared new sample shape: (1, 1, 19)


In [5]:

new_data = new_df.values

# Use the same scaler as before (fit on training data)
new_data_scaled = scaler.transform(new_data)  

# Reshape for CNN+LSTM model input: (samples, time steps, features)
new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))



C:\Users\deeks\anaconda3\envs\eeg_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [6]:
prediction = model.predict(new_data_reshaped)  # shape: (1, 2)

# Get predicted class (0 = Control, 1 = ADHD)
predicted_class = np.argmax(prediction, axis=1)[0]

if predicted_class == 1:
    print("Prediction: ADHD")
else:
    print("Prediction: Control")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
Prediction: Control
